In [1]:
from pathlib import Path
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

def plot_auc_result(data, category, execution, method):
    best_aucs = []
    for kfold, results in data.items():
        plotting = []
        for train_record in results[category]:
            df = pd.read_csv(Path.cwd() / execution / kfold / train_record)
            plotting.append((
                int(str(train_record.parent).replace("Epoch ", "")), 
                roc_auc_score(df["actual"], df["pred"])
            ))

        plotting.sort(key = lambda x: -x[1])
        best_aucs.append(plotting[0][1])
    return method(best_aucs)

def load_results(method):
    executions = set((Path.cwd().glob("results/complete-not-l2normed/*")))
    executions = executions - {
        Path.cwd() / "results/complete",
        Path.cwd() / "results/incomplete",
        Path.cwd() / "results/complete-not-l2normed"
    }

    df = {}

    for execution in tqdm(list(executions)):
        data = {}
        for folder in list(execution.glob("kfold-*")):
            data[str(folder.relative_to(execution))] = {
                "train": [i.relative_to(execution / folder) for i in folder.glob("Epoch */train-records.csv")],
                "test": [i.relative_to(execution / folder) for i in folder.glob("Epoch */test-records.csv")],
                # "eval": [i.relative_to(execution / folder) for i in folder.glob("Epoch */eval-records.csv")],
            }
        df[str(execution.relative_to(Path.cwd() / "results"))] = {
            "train": plot_auc_result(data, "train", execution, method),
            # "eval": plot_auc_result(data, "eval", execution, method),
            "test": plot_auc_result(data, "test", execution, method),
        }
    
    return pd.DataFrame(df).T.sort_index()


In [2]:
df = load_results(np.mean).sort_values("test")
df

  0%|          | 0/11 [00:00<?, ?it/s]

,train,test
complete-not-l2normed\atchley-no-reduction,0.761436,0.700014
complete-not-l2normed\tcrbert-autoencoder,0.753989,0.803922
complete-not-l2normed\sceptr-default-autoencoder,0.794407,0.817990
complete-not-l2normed\sceptr-tiny-autoencoder,0.814620,0.820328
complete-not-l2normed\aaprop-no-reduction,0.812252,0.846394
complete-not-l2normed\rand-no-reduction,0.877345,0.885740
complete-not-l2normed\kidera-no-reduction,0.758721,0.894001
complete-not-l2normed\kidera-autoencoder,0.848249,0.904929
complete-not-l2normed\sceptr-default-no-reduction,0.917568,0.934183
complete-not-l2normed\aaprop-autoencoder,0.854596,0.937543
